# Prism V2 Demo - K8

In [ ]:
%autoawait
import time
import re
import qrcode
import uuid
import base64
import jwt
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint
import json
import os

from dotenv import load_dotenv
load_dotenv()
APIKEY = os.getenv('APIKEY')

## Import Prism V2 Controller Library

In [ ]:
from prism_agent_open_api_specification_client import AuthenticatedClient

## Instantiate K8 Authenticated Agent Controllers

In [ ]:
from prism_agent_open_api_specification_client import AuthenticatedClient

issuer_client_controller = AuthenticatedClient(base_url="https://agent-sd67m.atalaprism.io/prism-agent", token=APIKEY)
holder_client_controller = AuthenticatedClient(base_url="https://agent-gr27n.atalaprism.io/prism-agent", token=APIKEY)
verifier_client_controller = AuthenticatedClient(base_url="https://agent-er56m.atalaprism.io/prism-agent", token=APIKEY)

## Import Connection Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

## Check Agent Connections

In [ ]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\nHolder Connections:\n{}\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

## Issuer Connects with Holder

In [ ]:
# Create Invitation
conn_request = CreateConnectionRequest()
conn_request.label = 'Connect with Holder'
issuer_invitation: Response[Connection] =  create_connection.sync(client=issuer_client_controller,json_body=conn_request)

print("Response:\n",issuer_invitation)
print("Connection ID\n",issuer_invitation.connection_id)
# print("Invitation URL\n",issuer_invitation.invitation.invitation_url)
issuer_invitationUrlbase64 = re.sub("^.*_oob=", "", issuer_invitation.invitation.invitation_url)
print("Invitation Base64\n",issuer_invitationUrlbase64)

## Holder Accepts Invitation from Issuer

In [ ]:
# Accept Invitation
accept_conn_request = AcceptConnectionInvitationRequest(issuer_invitationUrlbase64)
holder_invitation: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client_controller,json_body=accept_conn_request)
print(holder_invitation)

## Check Issuer and Holder Connection State

In [ ]:
# Check Connection State
issuer_conn: Response[Connection] =  get_connection.sync(client=issuer_client_controller,connection_id=issuer_invitation.connection_id)
holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)

while (issuer_conn.state != 'ConnectionResponseSent' and holder_conn.state != 'ConnectionResponseReceived'):
    issuer_conn: Response[Connection] =  get_connection.sync(client=issuer_client_controller,connection_id=issuer_invitation.connection_id)
    holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)
    print("Issuer State: {} Holder State: {} \n".format(issuer_conn.state,holder_conn.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")

## Check Agent Connections

In [ ]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\n\nHolder Connections:\n{}\n\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

## Import Credential Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.types import Response, Unset

## Check Credentials in Wallets

In [ ]:
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)
print("Issuer Credentials:\n{}\n\nHolder Credentials:\n{}\n\nVerifier Credentials:\n{}".format(issuer_creds, holder_creds,verifier_creds))

## Define Credential Schema and Content

In [ ]:
data = {
        "firstname": "Alice",
        "lastname": "Wonderland",
        "birthdate": "01/01/2000"
      }
cred_claims = CreateIssueCredentialRecordRequestClaims()
cred_claims = cred_claims.from_dict(data)
print(cred_claims)

credential_offer = CreateIssueCredentialRecordRequest(subject_id=issuer_conn.their_did, claims=cred_claims, schema_id="1234", validity_period=3600, automatic_issuance=False, await_confirmation=False)

print(credential_offer.to_dict())

## Send Credential Offer to Holder

In [ ]:
# Create Credential Offer
issuer_cred_offer: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client_controller,json_body=credential_offer)
print(issuer_cred_offer)

## Retrieve Credential Records from Issuer and Holder

In [ ]:
# Check Credential State
while (issuer_creds.items == [] or holder_creds.items == []):
    try:
            issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
            holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
            verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)
            
            time.sleep(1)
    except IndexError as e:
        pass

print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))

## Holder Accept Credential Offer from Issuer

In [ ]:
print(holder_creds.items[0].record_id)
holder_cred_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client_controller, record_id=holder_creds.items[0].record_id)
print(holder_cred_record)

## Issue Credential to Holder

In [ ]:
while holder_creds.items[0].protocol_state != 'RequestSent':
    holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)

issuer_cred_record: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client_controller, record_id=issuer_creds.items[0].record_id)
print(issuer_cred_record)

## Check if Credential is Successfully Issued to Holder

In [ ]:
# Check Credential State
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)

print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))

while (issuer_creds.items[0].protocol_state != 'CredentialSent' and holder_creds.items[0].protocol_state != 'CredentialReceived'):
    issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
    holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
    print("Issuer State: {} Holder State: {} \n".format(issuer_creds.items[0].protocol_state,holder_creds.items[0].protocol_state))
    time.sleep(1)
    
print(colored("Credential Record {} Issued to Holder! Continue with notebook".format(issuer_creds.items[0].record_id), "green", attrs=["bold"]))

### Decode JWT in Credential

In [ ]:
token = holder_creds.items[0].jwt_credential
jsondata = base64.urlsafe_b64decode(token)
b64decoded_token = json.loads(jsondata.decode())
print(b64decoded_token,'\n')
header = jwt.get_unverified_header(b64decoded_token)
print (header,'\n')

try:
    jwt_decoded_id_token = jwt.decode(b64decoded_token, options={"verify_signature": False})
    # print(jwt_decoded_id_token)
    print(json.dumps(jwt_decoded_id_token, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

In [ ]:
jwt_nbf = jwt_decoded_id_token['nbf']
jwt_exp = jwt_decoded_id_token['exp']
print('NBF epoch: {} EXP epoch: {}'.format(jwt_nbf,jwt_exp))

human_jwt_nbf = datetime.datetime.fromtimestamp(jwt_nbf)
human_jwt_exp = datetime.datetime.fromtimestamp(jwt_exp)
print('NBF: {} EXP: {}'.format(human_jwt_nbf,human_jwt_exp))

## Proof Presentation Flow

## Make a Connection between Holder and Verifier

## Check Agent Connections

In [ ]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\nHolder Connections:\n{}\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

## Verifier Connects with Holder

In [ ]:
# Create Invitation
conn_request = CreateConnectionRequest()
conn_request.label = 'Connect with Verifier'
verifier_invitation: Response[Connection] =  create_connection.sync(client=verifier_client_controller,json_body=conn_request)

print("Response:\n",verifier_invitation)
print("Connection ID\n",verifier_invitation.connection_id)
# print("Invitation URL\n",issuer_invitation.invitation.invitation_url)
verifier_invitationUrlbase64 = re.sub("^.*_oob=", "", verifier_invitation.invitation.invitation_url)
print("Invitation Base64\n",verifier_invitationUrlbase64)

## Holder Accepts Invitation from Verifier

In [ ]:
# Accept Invitation
accept_conn_request = AcceptConnectionInvitationRequest(verifier_invitationUrlbase64)
holder_invitation: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client_controller,json_body=accept_conn_request)
print(holder_invitation)

## Check Verifier and Holder Connection State

In [ ]:
# Check Connection State
verifier_conn: Response[Connection] =  get_connection.sync(client=verifier_client_controller,connection_id=verifier_invitation.connection_id)
holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)

while (verifier_conn.state != 'ConnectionResponseSent' and holder_conn.state != 'ConnectionResponseReceived'):
    verifier_conn: Response[Connection] =  get_connection.sync(client=verifier_client_controller,connection_id=verifier_invitation.connection_id)
    holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)
    print("Verifier State: {} Holder State: {} \n".format(verifier_conn.state,holder_conn.state))
    time.sleep(1)
    
print("Connection established between Verifier and Holder!")

## Import Preentation Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import PresentationStatus, ErrorResponse, Proof, ProofRequestAux, PublicKeyJwk, RequestPresentationInput, RequestPresentationOutput, RequestPresentationAction
from prism_agent_open_api_specification_client.api.present_proof import get_all_presentation, request_presentation, update_presentation
from prism_agent_open_api_specification_client.types import Response

## Check Presentation Requests in Wallets

In [ ]:
issuer_proofs: Response[Proof] = get_all_presentation.sync(client=issuer_client_controller)
holder_proofs: Response[Proof] = get_all_presentation.sync(client=holder_client_controller)
verifier_proofs: Response[Proof] = get_all_presentation.sync(client=verifier_client_controller)
print("Issuer Proofs:\n{}\n\nHolder Proofs:\n{}\n\nVerifier Proofs:\n{}".format(issuer_proofs, holder_proofs,verifier_proofs))

## Initiate a Proof Request

In [ ]:
data = {
  "connectionId": verifier_invitation.connection_id, "proofs":[]
}
proof_request = RequestPresentationInput.from_dict(data)

verifier_proof_request: Response[RequestPresentationInput] = request_presentation.sync(client=verifier_client_controller, json_body=proof_request)

print(verifier_proof_request)

## Check Presentation Requests in Wallets

In [ ]:
while holder_proofs == []:
    issuer_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=issuer_client_controller)
    holder_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=holder_client_controller)
    verifier_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=verifier_client_controller)
print("Issuer Proofs:\n{}\n\nHolder Proofs:\n{}\n\nVerifier Proofs:\n{}".format(issuer_proofs, holder_proofs,verifier_proofs))

### Accepting the Presentation Request

In [ ]:
from typing import Any, Dict, Optional, Union, cast

data = {
  "action": "request-accept",
  "proofId": [holder_creds.items[0].record_id]
}

proof_accept = RequestPresentationAction.from_dict(data)

holder_proof_request: Response[Union[Any, ErrorResponse]] = update_presentation.sync(client=holder_client_controller, json_body=proof_accept, id=holder_proofs[0].presentation_id)

print(holder_proof_request)

## Check Proof Presentation State

In [ ]:
# Check Proof Presentation State
while (holder_proofs[0].status != 'PresentationSent' and verifier_proofs[0].status != 'PresentationVerified'):
    holder_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=holder_client_controller)
    verifier_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=verifier_client_controller)
    print("Verifier State: {} Holder State: {} \n".format(holder_proofs[0].status,verifier_proofs[0].status))
    time.sleep(1)
    
print("Proof Presentation completed between Verifier and Holder!")

## Check Presentation Requests in Wallets

In [ ]:
issuer_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=issuer_client_controller)
holder_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=holder_client_controller)
verifier_proofs: Response[PresentationStatus] = get_all_presentation.sync(client=verifier_client_controller)
print("Issuer Proofs:\n{}\n\nHolder Proofs:\n{}\n\nVerifier Proofs:\n{}".format(issuer_proofs, holder_proofs,verifier_proofs))

### Decode JWT in Credential

In [ ]:
print(loaded_json)

In [ ]:
token = holder_creds.items[0].jwt_credential
jsondata = base64.urlsafe_b64decode(token)
b64decoded_token = json.loads(jsondata.decode())
print(b64decoded_token,'\n')
header = jwt.get_unverified_header(b64decoded_token)
print (header,'\n')

try:
    jwt_decoded_id_token = jwt.decode(b64decoded_token, options={"verify_signature": False})
    # print(jwt_decoded_id_token)
    print(json.dumps(jwt_decoded_id_token, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

In [ ]:
jwt_nbf = jwt_decoded_id_token['nbf']
jwt_exp = jwt_decoded_id_token['exp']
print('NBF epoch: {} EXP epoch: {}'.format(jwt_nbf,jwt_exp))

human_jwt_nbf = datetime.datetime.fromtimestamp(jwt_nbf)
human_jwt_exp = datetime.datetime.fromtimestamp(jwt_exp)
print('NBF: {} EXP: {}'.format(human_jwt_nbf,human_jwt_exp))

## DIDs

## Import DID Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import DID, DIDDocumentMetadata, DIDOperationResponse, DIDResponse, CreateManagedDidRequest, ListManagedDIDResponseInner, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.api.did import get_did
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did,list_managed_did,publish_managed_did
from prism_agent_open_api_specification_client.types import Response

### List DIDs in Wallet

In [ ]:
issuer_dids: Response[ListManagedDIDResponseInner] = list_managed_did.sync(client=issuer_client_controller)
print(issuer_dids)
print(issuer_dids[0].did)

### Create DID

In [ ]:
data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }
new_did = CreateManagedDidRequest.from_dict(data)

issuer_new_did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=issuer_client_controller, json_body=new_did)
print(issuer_new_did)

### Resolve DID

In [ ]:
issuer_resolve_did: Response[DID] = get_did.sync(client=issuer_client_controller, did_ref=issuer_new_did.long_form_did)
print(issuer_resolve_did)

issuer_resolve_did: Response[DID] = get_did.sync(client=issuer_client_controller, did_ref=issuer_dids[0].did)
print(issuer_resolve_did)


### Publish DID

In [ ]:
issuer_publish_did: Response[DID] = publish_managed_did.sync(client=issuer_client_controller, did_ref=issuer_new_did.long_form_did)
print(issuer_publish_did)

### Deactivate DID